# Whole-brain Connective Field mapping

In [ ]:
# Safe imports with error handling
import sys
import warnings
warnings.filterwarnings('ignore')

# Core imports
try:
    import os
    import glob
    import yaml
    from pathlib import Path
    import pickle
    import numpy as np
    import nibabel as nib
    print('✅ Core imports successful')
except Exception as e:
    print(f'❌ Core imports failed: {e}')
    raise

# Neuroimaging imports
try:
    import neuropythy as ny
    from neuropythy.geometry import Mesh, Tesselation
    print('✅ Neuropythy loaded')
except Exception as e:
    print(f'❌ Neuropythy failed: {e}')
    raise

# Visualization imports
try:
    import pandas as pd
    import ipyvolume as ipv
    import matplotlib.pyplot as plt
    from matplotlib.patches import Patch, Wedge
    import matplotlib.colors as mcolors
    from matplotlib import cm
    from mpl_toolkits.axes_grid1.inset_locator import inset_axes
    print('✅ Visualization imports successful')
except Exception as e:
    print(f'❌ Visualization imports failed: {e}')
    raise

# Additional scientific imports
try:
    from nilearn.surface import vol_to_surf
    from nilearn import surface, plotting, signal
    from scipy.spatial.distance import pdist, squareform
    from scipy.spatial import cKDTree
    from scipy.stats import pearsonr
    print('✅ Scientific imports successful')
except Exception as e:
    print(f'⚠️ Some scientific imports failed: {e}')

# Widget imports
try:
    from ipywidgets import FloatText, HBox, VBox, Textarea, Output, Dropdown, FloatSlider, interactive_output
    from traitlets import link
    from IPython.display import display
    print('✅ Widget imports successful')
except Exception as e:
    print(f'❌ Widget imports failed: {e}')
    raise


import math
import gc

print('\n✅ All critical imports completed!')


In [ ]:
# Inline color palettes (no external cfmap dependency)
def get_color_palettes():
    """Generate color palettes for visualization."""
    from matplotlib.colors import ListedColormap, LinearSegmentedColormap
    
    # Eccentricity: 15 colors from red to pink
    eccen_hex = [
        '#FF0000', '#FF3300', '#FF6600', '#FF9900', '#FFCC00',
        '#CCFF00', '#99FF00', '#66FF00', '#33FF00', '#00FF00',
        '#00FF99', '#00FFFF', '#0099FF', '#0033FF', '#FF00FF'
    ]
    
    # Polar: 20-color circular wheel
    polar_hex = [
        '#FF0000', '#FF4000', '#FF8000', '#FFBF00', '#FFFF00',
        '#BFFF00', '#80FF00', '#40FF00', '#00FF00', '#00FF40',
        '#00FF80', '#00FFBF', '#00FFFF', '#00BFFF', '#0080FF',
        '#0040FF', '#0000FF', '#4000FF', '#8000FF', '#BF00FF'
    ]
    
    return {
        'eccentricity': {
            'hex': eccen_hex,
            'matplotlib_cmap': ListedColormap(eccen_hex, name='eccen')
        },
        'polar': {
            'hex': polar_hex,
            'matplotlib_cmap': ListedColormap(polar_hex, name='polar')
        }
    }

color_palettes = get_color_palettes()
eccen_colors = color_palettes["eccentricity"]
polar_colors = color_palettes["polar"]


# Rotate axis
def rotate_coords(coords, axis, angle_degrees):
    """
    Rotates coordinates by a given angle around the specified axis.
    
    Parameters:
        coords (np.ndarray): shape (3, N) (x, y, z as first dimension)
        axis (str): 'x', 'y', or 'z'
        angle_degrees (float): rotation angle in degrees
        
    Returns:
        np.ndarray: rotated coordinates, shape (3, N)
    """
    theta = np.deg2rad(angle_degrees)
    if axis == 'x':
        rot = np.array([
            [1, 0, 0],
            [0, np.cos(theta), -np.sin(theta)],
            [0, np.sin(theta),  np.cos(theta)]
        ])
    elif axis == 'y':
        rot = np.array([
            [ np.cos(theta), 0, np.sin(theta)],
            [ 0,             1, 0            ],
            [-np.sin(theta), 0, np.cos(theta)]
        ])
    elif axis == 'z':
        rot = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta),  np.cos(theta), 0],
            [0,              0,             1]
        ])
    else:
        raise ValueError("axis must be 'x', 'y', or 'z'")
    return rot @ coords



# Plotting function (adapted from original)
def plot_and_save_brains(lh_map, rh_map, colormap, mesh_lh, mesh_rh, strips_lh, strips_rh, mask_lh, mask_rh, view, vmin=None, vmax=None, cbar_label='Value', cf_property='r2', polar_colormap='polar'):
    """
    Plot brain surfaces with given maps and colormap, set the view based on the flag, and save to PNG.
    
    Parameters:
    - lh_map: array-like, map data for left hemisphere
    - rh_map: array-like, map data for right hemisphere
    - colormap: matplotlib colormap object
    - mesh_lh: mesh object for left hemisphere
    - mesh_rh: mesh object for right hemisphere
    - strips_lh: underlay data for left hemisphere
    - strips_rh: underlay data for right hemisphere
    - mask_lh: mask for left hemisphere
    - mask_rh: mask for right hemisphere
    - view: str ('ventral' or 'dorsal') or tuple (azim, elev, dist) to set the camera view
    - vmin: float, minimum value for colormap scaling (optional)
    - vmax: float, maximum value for colormap scaling (optional)
    - cbar_label: str, label for the colorbar (optional, default: 'Value')
    - cf_property: str, CF property being plotted (for specialized colorbar insets)
    - polar_colormap: str, colormap type for polar angle ('polar' or 'hsv')
    """
    if isinstance(view, tuple) and len(view) == 3:
        azim, elev, dist = view
    elif view == 'ventral':
        azim, elev, dist = -172, -8, 180
    elif view == 'dorsal':
        azim, elev, dist = -6.13, 31.34, 46.26
    else:
        raise ValueError("view must be 'ventral', 'dorsal', or a tuple (azim, elev, dist)")
       
    # Create figure with default size (640x480)
    fig = ipv.figure(width=640, height=480)
    
    # Plot right hemisphere
    ny.cortex_plot(mesh_rh, surface='inflated', color=rh_map, cmap=colormap,
        underlay=strips_rh, underlay_cmap='gray', underlay_vmin=-5, underlay_vmax=0.0, mask=mask_rh,
        vmin=vmin, vmax=vmax,
        figure=fig)
    
    # Plot left hemisphere
    ny.cortex_plot(mesh_lh, surface='inflated', color=lh_map, cmap=colormap,
        underlay=strips_lh, underlay_cmap='gray', underlay_vmin=-5, underlay_vmax=0.0, mask=mask_lh,
        vmin=vmin, vmax=vmax,
        figure=fig)
    
    # Compute the center of the plot (mean of all mesh coordinates)
    all_coords = np.concatenate([mesh_lh.coordinates, mesh_rh.coordinates], axis=1)
    center = np.mean(all_coords, axis=1)
    fig.camera.center = center
    
    # Custom function to set view relative to center
    def set_view(fig, azimuth, elevation, distance):
        center = fig.camera.center
        elev_rad = np.radians(elevation)
        az_rad = np.radians(azimuth)
        unit = np.array([
            np.cos(elev_rad) * np.cos(az_rad),
            np.cos(elev_rad) * np.sin(az_rad),
            np.sin(elev_rad)
        ])
        fig.camera.position = tuple(center + distance * unit)
    
    # Adjust the final view
    set_view(fig, azim, elev, dist)

    ipv.show()
    
    # Add colorbar using inset - property-specific
    import matplotlib.pyplot as plt
    from matplotlib import cm
    from mpl_toolkits.axes_grid1.inset_locator import inset_axes
    from matplotlib.patches import Wedge
    
    # Normalize values
    vmin_val = vmin if vmin is not None else np.nanmin([np.nanmin(lh_map), np.nanmin(rh_map)])
    vmax_val = vmax if vmax is not None else np.nanmax([np.nanmax(lh_map), np.nanmax(rh_map)])
    
    if cf_property == 'eccentricity':
        # Create radial concentric rings colorbar for eccentricity
        fig_cb, ax_main = plt.subplots(figsize=(3, 3))
        ax_main.set_aspect('equal')
        ax_main.set_xlim(-1.5, 1.5)
        ax_main.set_ylim(-1.5, 1.5)
        ax_main.set_axis_off()
        ax_main.text(0.5, -0.05, r'$\mathit{r}\ (\mathrm{deg})$', ha='center', va='top', 
                    fontsize=14, transform=ax_main.transAxes)
        
        num_ecc_colors = len(eccen_colors["hex"])
        for i, color in enumerate(eccen_colors["hex"]):
            inner_r = i / num_ecc_colors
            outer_r = (i + 1) / num_ecc_colors
            ring = Wedge((0, 0), outer_r, 0, 360, width=outer_r - inner_r, color=color)
            ax_main.add_patch(ring)
        
        plt.tight_layout()
        plt.show()
        
    elif cf_property == 'polar':
        # Create polar pie chart colorbar for polar angle
        fig_cb, ax_main = plt.subplots(figsize=(3, 3))
        ax_main.set_aspect('equal')
        ax_main.set_axis_off()
        
        if polar_colormap == 'hsv':
            # Use HSV colormap - create gradient pie chart
            import matplotlib.pyplot as plt
            n_segments = 20
            theta = np.linspace(0, 2*np.pi, n_segments, endpoint=False)
            colors_hsv = [plt.cm.hsv(i/n_segments) for i in range(n_segments)]
            ax_main.pie([1]*n_segments, colors=colors_hsv, 
                       startangle=180, counterclock=False)
        else:
            # Use custom polar colormap
            ax_main.pie([1]*len(polar_colors["hex"]), colors=polar_colors["hex"], 
                       startangle=180, counterclock=False)
        
        ax_main.text(0.5, -0.05, r'$\theta\ (\mathrm{rad})$', ha='center', va='top', 
                    fontsize=14, transform=ax_main.transAxes)
        
        plt.tight_layout()
        plt.show()
        
    else:
        # Create standard horizontal colorbar for other properties
        fig_cb, ax_main = plt.subplots(figsize=(8, 2))
        ax_main.set_axis_off()
        
        # Create inset for horizontal colorbar
        cbar_inset = inset_axes(ax_main, width="70%", height="30%", loc="center", borderpad=0)
        
        norm = plt.Normalize(vmin=vmin_val, vmax=vmax_val)
        
        # Create colorbar in inset
        cb = plt.colorbar(cm.ScalarMappable(norm=norm, cmap=colormap),
                          cax=cbar_inset, orientation='horizontal')
        cb.set_label(cbar_label, fontsize=14)
        cb.ax.tick_params(labelsize=12)
        
        plt.tight_layout()
        plt.show()

    # Create widgets for real-time updates
    azimuth_widget = FloatText(description='Azimuth:', step=0.1, disabled=True)
    elevation_widget = FloatText(description='Elevation:', step=0.1, disabled=True)
    distance_widget = FloatText(description='Distance:', step=0.1, disabled=True)

    set_view_widget = Textarea(
        description='set_view call:',
        value='set_view(fig, 0.00, 0.00, 0.00)',
        disabled=True,
        layout={'width': '400px', 'height': '50px'}
    )

    def update_widgets(change):
        pos = fig.camera.position
        center = fig.camera.center
        v = np.array(pos) - np.array(center)
        dist = np.linalg.norm(v)
        if dist > 0:
            elevation = np.degrees(np.arcsin(v[2] / dist))
            azimuth = np.degrees(np.arctan2(v[1], v[0]))
        else:
            azimuth = 0
            elevation = 0
        distance = dist
        azimuth_widget.value = azimuth
        elevation_widget.value = elevation
        distance_widget.value = distance
        set_view_widget.value = f"set_view(fig, {azimuth:.2f}, {elevation:.2f}, {distance:.2f})"
        print(set_view_widget.value)

    fig.camera.observe(update_widgets, names=['position'])
    update_widgets(None)

    # Display the widgets
    from IPython.display import display
    display(VBox([HBox([azimuth_widget, elevation_widget, distance_widget]), set_view_widget]))


In [ ]:
# Interactive CF Results Plotting
# Set anterior_threshold to None for full whole brain, or a value (e.g., -30) for posterior cut
anterior_threshold = None
from ipywidgets import Dropdown, FloatSlider, interact
import glob
import re
from pathlib import Path

# Import necessary libraries
import numpy as np

# Get the current notebook's directory and define paths
notebook_dir = Path().resolve()

# Try multiple possible data locations
possible_paths = [
    notebook_dir / 'data' / 'output',
    notebook_dir.parent / 'output',
    Path('/media/ng281432/Crucial X6/UNICOG/LePetitePrince/output')
]

base_data_path = None
for path in possible_paths:
    if path.exists():
        base_data_path = path
        break

if base_data_path is None:
    print("⚠️ No data found. Please ensure data is available.")
    print("Expected paths:")
    for p in possible_paths:
        print(f"  - {p}")
    # Create dummy structure for demonstration
    base_data_path = notebook_dir / 'data' / 'output'
    base_data_path.mkdir(parents=True, exist_ok=True)
    
print(f"Base data path: {base_data_path}")


parent_path = glob.glob(str(base_data_path.parent), recursive=True)
parent_path = Path(parent_path[0])
print(f"Parent path: {parent_path}")

# Define CF models directory and FreeSurfer subjects directory
cf_models_dir = base_data_path / 'derivatives' / 'cf-models'
fs_subjects_dir = base_data_path / 'fs_subjects'

print(f"CF models directory: {cf_models_dir}")
print(f"FreeSurfer subjects directory: {fs_subjects_dir}")

# Scan available CF model files
def scan_cf_models(cf_models_dir):
    """Scan CF models directory and extract available subjects, tasks, and sources."""
    cf_models_dir = Path(cf_models_dir)
    pattern = str(cf_models_dir / 'cf_results_sub-*.npz')
    files = glob.glob(pattern)
    
    available_models = []
    subjects = set()
    tasks = set()
    
    for f in files:
        basename = Path(f).name
        # Parse: cf_results_sub-01_ses-01_LPP1_lh-source_ecc0.5-20.0.npz
        match = re.match(r'cf_results_sub-(\d+)_(ses-\d+)_(\w+)_(lh|rh)-source_ecc([\d.]+)-([\d.]+)\.npz', basename)
        if match:
            subject_id, session_id, task, source_hemi, min_ecc, max_ecc = match.groups()
            subjects.add(subject_id)
            tasks.add(task)
            available_models.append({
                'file': f,
                'subject': subject_id,
                'session': session_id,
                'task': task,
                'source_hemi': source_hemi,
                'min_ecc': float(min_ecc),
                'max_ecc': float(max_ecc)
            })
    
    return sorted(list(subjects)), sorted(list(tasks)), available_models

# Scan models
available_subjects, available_tasks, cf_models_info = scan_cf_models(cf_models_dir)

print(f"Found {len(cf_models_info)} CF model files")
print(f"Subjects: {available_subjects}")
print(f"Tasks: {available_tasks}")

# Define property-specific defaults
property_config = {
    'r2': {
        'adaptive': True,
        'vmin': 0,
        'vmax': 1
    },
    'polar': {
        'adaptive': True,
        'vmin': -np.pi,
        'vmax': np.pi
    },
    'eccentricity': {
        'adaptive': False,
        'vmin': 0.5,
        'vmax': 6.5
    },
    'cf_size': {
        'adaptive': False,
        'vmin': 0.5,
        'vmax': 5.0
    }
}

# Cache for loaded results
loaded_results_cache = {}

def load_cf_results(subject_id, task, source_hemi):
    """Load CF results for given subject, task, and source hemisphere."""
    # Create cache key
    cache_key = f"sub-{subject_id}_{task}_{source_hemi}"
    
    # Check cache
    if cache_key in loaded_results_cache:
        return loaded_results_cache[cache_key]
    
    # Find matching file
    matching_files = [m for m in cf_models_info 
                     if m['subject'] == subject_id 
                     and m['task'] == task 
                     and m['source_hemi'] == source_hemi]
    
    if not matching_files:
        print(f"❌ No model found for sub-{subject_id}, task={task}, source={source_hemi}")
        return None, None
    
    model_file = matching_files[0]['file']
    print(f"📂 Loading: {Path(model_file).name}")
    
    # Load results
    results = np.load(model_file, allow_pickle=True)
    results_lh = results['lh_results'].item()
    results_rh = results['rh_results'].item()
    
    # Cache results
    loaded_results_cache[cache_key] = (results_lh, results_rh)
    
    return results_lh, results_rh

# Function to update plot based on selected parameters
def update_plot(subject_id, task, source_hemi, cf_property, r2_threshold, use_adaptive_range, vmin, vmax, polar_colormap='polar'):
    # Load results for selected subject/task/source
    results_lh, results_rh = load_cf_results(subject_id, task, source_hemi)
    
    if results_lh is None or results_rh is None:
        print("❌ Failed to load results")
        return

    # Load subject mesh (update if subject changes)
    global sub, lh_mesh, rh_mesh, lh_curv_map, rh_curv_map, current_loaded_subject
    current_sub_id = f"{int(subject_id):02d}"
    
    # Check if we need to load a new subject
    if 'current_loaded_subject' not in globals() or current_loaded_subject != current_sub_id:
        # FreeSurfer subject directory format: sub-01_ses-01_iso
        fs_subject_name = f"sub-{current_sub_id}_ses-01_iso"
        fs_subject_path = fs_subjects_dir / fs_subject_name
        
        print(f"🧠 Loading FreeSurfer subject: {fs_subject_name}")
        print(f"   Path: {fs_subject_path}")
        
        if not fs_subject_path.exists():
            print(f"❌ FreeSurfer subject not found: {fs_subject_path}")
            return
        
        # Load surfaces directly using nibabel (faster, no full FS structure needed)
        try:
            import nibabel.freesurfer as fs
            surf_dir = fs_subject_path / 'surf'
            
            # Load geometry
            lh_coords, lh_faces = fs.read_geometry(str(surf_dir / 'lh.inflated'))
            rh_coords, rh_faces = fs.read_geometry(str(surf_dir / 'rh.inflated'))
            
            # Create mesh objects (neuropythy expects coords as (3, n_vertices))
            lh_mesh = Mesh(Tesselation(lh_faces.T), lh_coords.T)
            rh_mesh = Mesh(Tesselation(rh_faces.T), rh_coords.T)
            
            # Load curvature
            lh_curv_map = fs.read_morph_data(str(surf_dir / 'lh.curv'))
            rh_curv_map = fs.read_morph_data(str(surf_dir / 'rh.curv'))
            
            print(f"   ✅ Loaded: LH={len(lh_coords)} vertices, RH={len(rh_coords)} vertices")
            
        except Exception as e:
            print(f"❌ Error loading surfaces: {e}")
            import traceback
            traceback.print_exc()
            return
        current_loaded_subject = current_sub_id

    # Select data based on property
    prop_dict = {
        'eccentricity': 'inherited_eccen',
        'polar': 'inherited_polar',
        'cf_size': 'cf_size',
        'r2': 'r2'
    }
    if cf_property not in prop_dict:
        print(f"Unknown property: {cf_property}")
        return

    lh_data = results_lh[prop_dict[cf_property]].copy()
    rh_data = results_rh[prop_dict[cf_property]].copy()
    
    # Apply R² threshold - mask out voxels below threshold OR with NaN R²
    lh_r2 = results_lh['r2']
    rh_r2 = results_rh['r2']
    
    # Create comprehensive mask: mask where R² is NaN OR below threshold
    lh_mask = np.isnan(lh_r2) | (lh_r2 < r2_threshold)
    rh_mask = np.isnan(rh_r2) | (rh_r2 < r2_threshold)
    
    lh_data[lh_mask] = np.nan
    rh_data[rh_mask] = np.nan
    
    n_valid_lh = np.sum(~np.isnan(lh_data))
    n_valid_rh = np.sum(~np.isnan(rh_data))
    print(f"R² threshold: {r2_threshold:.2f} | Valid vertices: LH={n_valid_lh}, RH={n_valid_rh}")

    # Calculate adaptive vmin/vmax based on actual data (override sliders if adaptive is True)
    if use_adaptive_range:
        # Get valid (non-NaN) data from both hemispheres
        valid_data = np.concatenate([
            lh_data[~np.isnan(lh_data)],
            rh_data[~np.isnan(rh_data)]
        ])
        if len(valid_data) > 0:
            vmin = np.percentile(valid_data, 2)   # 2nd percentile to avoid outliers
            vmax = np.percentile(valid_data, 98)  # 98th percentile to avoid outliers
            print(f"Adaptive range for {cf_property}: [{vmin:.3f}, {vmax:.3f}]")
        else:
            # Fallback to config defaults
            config = property_config.get(cf_property, {'vmin': 0, 'vmax': 1})
            vmin, vmax = config['vmin'], config['vmax']
    else:
        print(f"Manual range for {cf_property}: [{vmin:.3f}, {vmax:.3f}]")

    # Prepare for plotting (full brain)
    lh_grad_plot = lh_data
    rh_grad_plot = rh_data
    lh_strips_plot = lh_curv_map.astype(float)
    lh_strips_plot[lh_strips_plot > 0] = np.nan
    rh_strips_plot = rh_curv_map.astype(float)
    rh_strips_plot[rh_strips_plot > 0] = np.nan
    lh_mask_plot = ~np.isnan(lh_data)
    rh_mask_plot = ~np.isnan(rh_data)

    # Apply rotations and shifts
    angle = 0
    lh_coords_plot = lh_mesh.coordinates
    rh_coords_plot = rh_mesh.coordinates
    lh_faces_plot = lh_mesh.tess.faces
    rh_faces_plot = rh_mesh.tess.faces
    lh_coords_medial = rotate_coords(lh_coords_plot, axis='z', angle_degrees=-angle)
    rh_coords_medial = rotate_coords(rh_coords_plot, axis='z', angle_degrees=angle*2)
    rh_coords_medial[0, :] += 80
    lh_mesh_shifted = Mesh(Tesselation(lh_faces_plot), lh_coords_medial)
    rh_mesh_shifted = Mesh(Tesselation(rh_faces_plot), rh_coords_medial)

    # Select colormap based on property
    if cf_property == 'eccentricity':
        grad_cmap = eccen_colors['matplotlib_cmap']
        cbar_label = r'Eccentricity $r$ (deg)'
    elif cf_property == 'polar':
        # Use selected colormap for polar angle
        if polar_colormap == 'hsv':
            grad_cmap = plt.colormaps['hsv']
        else:
            grad_cmap = polar_colors['matplotlib_cmap']
        cbar_label = r'Polar angle $\theta$ (rad)'
    elif cf_property == 'r2':
        grad_cmap = plt.colormaps['viridis']
        cbar_label = r'$R^2$'
    elif cf_property == 'cf_size':
        grad_cmap = plt.colormaps['viridis']
        cbar_label = r'CF size $\sigma$ (mm)'
    else:
        grad_cmap = plt.colormaps['viridis']
        cbar_label = 'Value'

    # Plot (no interpolation) - pass cf_property and polar_colormap for specialized colorbar
    plot_and_save_brains(lh_grad_plot, rh_grad_plot, grad_cmap,
                         lh_mesh_shifted, rh_mesh_shifted,
                         lh_strips_plot, rh_strips_plot,
                         lh_mask_plot, rh_mask_plot,
                         (-122, -27, 80), vmin=vmin, vmax=vmax, cbar_label=cbar_label,
                         cf_property=cf_property, polar_colormap=polar_colormap)
    
    # Store current plot data for export functionality
    global current_plot_data
    current_plot_data = {
        'lh_data': lh_grad_plot,
        'rh_data': rh_grad_plot,
        'lh_mesh': lh_mesh_shifted,
        'rh_mesh': rh_mesh_shifted,
        'colormap': grad_cmap,
        'vmin': vmin,
        'vmax': vmax,
        'cbar_label': cbar_label,
        'subject_id': subject_id,
        'task': task,
        'source_hemi': source_hemi,
        'cf_property': cf_property,
        'r2_threshold': r2_threshold
    }

# Create widgets
subject_widget = Dropdown(options=available_subjects, 
                         value=available_subjects[0] if available_subjects else '01', 
                         description='Subject:')
task_widget = Dropdown(options=available_tasks, 
                      value=available_tasks[0] if available_tasks else 'LPP1', 
                      description='Task:')
source_hemi_widget = Dropdown(options=['lh', 'rh'], value='lh', description='Source:')
cf_property_widget = Dropdown(options=['eccentricity', 'polar', 'cf_size', 'r2'], 
                              value='eccentricity', description='CF parameter:')
r2_threshold_widget = FloatSlider(value=0.1, min=0.0, max=1.0, step=0.01, description='R² threshold:')

# Adaptive range widget - default depends on CF property
adaptive_range_widget = Dropdown(options=[True, False], value=False, 
                                description='Range:')

# Min/max widgets - defaults depend on CF property - initialized for eccentricity
vmin_widget = FloatSlider(value=0.5, min=-10, max=10, step=0.01, description='min:')
vmax_widget = FloatSlider(value=6.5, min=-10, max=10, step=0.01, description='max:')

# Polar colormap widget - only visible when plotting polar angle
polar_colormap_widget = Dropdown(options=['polar', 'hsv'], value='polar', 
                                 description='Polar cmap:')

# Function to update widget defaults when CF property changes
def update_widget_defaults(cf_property):
    """Update adaptive range and min/max defaults based on CF property."""
    config = property_config.get(cf_property, {'adaptive': True, 'vmin': 0, 'vmax': 1})
    adaptive_range_widget.value = config['adaptive']
    vmin_widget.value = config['vmin']
    vmax_widget.value = config['vmax']

# Link CF property widget to update defaults
cf_property_widget.observe(lambda change: update_widget_defaults(change['new']), names='value')

# Interactive plot with conditional polar colormap widget
from ipywidgets import interactive_output, VBox, HBox
from IPython.display import display

# Create interactive output
ui_controls = {
    'subject_id': subject_widget,
    'task': task_widget, 
    'source_hemi': source_hemi_widget,
    'cf_property': cf_property_widget,
    'r2_threshold': r2_threshold_widget,
    'use_adaptive_range': adaptive_range_widget,
    'vmin': vmin_widget,
    'vmax': vmax_widget,
    'polar_colormap': polar_colormap_widget
}

out = interactive_output(update_plot, ui_controls)

# Function to update widget visibility based on CF property
def update_widget_visibility(change):
    """Show polar colormap widget only when polar angle is selected."""
    if change['new'] == 'polar':
        polar_colormap_widget.layout.display = 'flex'
    else:
        polar_colormap_widget.layout.display = 'none'

# Initially hide polar colormap widget if not plotting polar
if cf_property_widget.value != 'polar':
    polar_colormap_widget.layout.display = 'none'

# Link visibility to CF property changes
cf_property_widget.observe(update_widget_visibility, names='value')

# Display widgets and output
display(VBox([
    subject_widget,
    task_widget,
    source_hemi_widget,
    cf_property_widget,
    r2_threshold_widget,
    adaptive_range_widget,
    vmin_widget,
    vmax_widget,
    polar_colormap_widget,
    out
]))



Base data path: /media/ng281432/Crucial X6/UNICOG/LePetitePrince/output
Parent path: /media/ng281432/Crucial X6/UNICOG/LePetitePrince
CF models directory: /media/ng281432/Crucial X6/UNICOG/LePetitePrince/output/derivatives/cf-models
FreeSurfer subjects directory: /media/ng281432/Crucial X6/UNICOG/LePetitePrince/output/fs_subjects
Found 7 CF model files
Subjects: ['01']
Tasks: ['LPP1', 'LPP2', 'LPP3', 'LPP4']
